In [2]:
import numpy as np
from matplotlib import pyplot

Extending the 1RM calculator

Many are familiar with the idea of one rep max (1RM) calculators, where a weight and a number of reps performed is used to calculate the maximum weight the trainee could lift. Perhaps the most popular formula is the Epley formula:  
$$1\mathrm{RM} = 30 \cdot \left( 1 + \frac{\mathrm{reps}}{30} \right)$$
In many cases, this is reasonably good. Few would call it perfect however. The main drawback is simply that the repetition strength varies with individuals as well as with